<a href="https://colab.research.google.com/github/SiThuTun22/ML-Projects/blob/main/HarmFulDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone YOLOv5 repo and install requirements
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17516, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 17516 (delta 8), reused 0 (delta 0), pack-reused 17494 (from 3)
Receiving objects: 100% (17516/17516), 16.66 MiB | 25.84 MiB/s, done.
Resolving deltas: 100% (12000/12000), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta

In [ ]:
from google.colab import files
files.upload()  # Upload your kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sithutungraki","key":"33d8c49dcaadf3bbb5efd43a75da69b8"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d raghavnanjappan/weapon-dataset-for-yolov5
!unzip weapon-dataset-for-yolov5.zip -d weapons-dataset

Streaming output truncated to the last 5000 lines.
  inflating: weapons-dataset/dataset/dataset/images/train/4051.jpg  
  inflating: weapons-dataset/dataset/dataset/images/train/4052.jpg  
  inflating: weapons-dataset/dataset/dataset/images/train/4053.jpg  
  inflating: weapons-dataset/dataset/dataset/images/train/4054.jpg  
  inflating: weapons-dataset/dataset/dataset/images/train/4055.jpg  
  inflating: weapons-dataset/dataset/dataset/images/train/4056.jpg  
  inflating: weapons-dataset/dataset/dataset/images/train/4057.jpg  
  inflating: weapons-dataset/dataset/dataset/images/train/4058.jpg  
  inflating: weapons-dataset/dataset/dataset/images/train/4059.jpg  
  inflating: weapons-dataset/dataset/dataset/images/train/406.jpg  
  inflating: weapons-dataset/dataset/dataset/images/train/4060.jpg  
  inflating: weapons-dataset/dataset/dataset/images/train/4061.jpg  
  inflating: weapons-dataset/dataset/dataset/images/train/4062.jpg  
  inflating: weapons-dataset/dataset/dataset/images/t

In [ ]:
!mkdir -p /content/weapons-dataset/images/train
!mkdir -p /content/weapons-dataset/images/val
!mkdir -p /content/weapons-dataset/labels/train
!mkdir -p /content/weapons-dataset/labels/val


In [ ]:
import os
import random
import shutil

# Paths
source_img_dir = "/content/yolov5/weapons-dataset/dataset/dataset/images/train"
source_lbl_dir = "/content/yolov5/weapons-dataset/dataset/dataset/labels/train"
target_dir = "/content/weapons-dataset"

# Get file list and shuffle
all_files = os.listdir(source_img_dir)
random.shuffle(all_files)

# Split
split_idx = int(len(all_files) * 0.8)
train_files = all_files[:split_idx]
val_files = all_files[split_idx:]

# Helper to move files
def move_files(file_list, subset):
    for file in file_list:
        base = os.path.splitext(file)[0]
        img_src = os.path.join(source_img_dir, file)
        lbl_src = os.path.join(source_lbl_dir, base + ".txt")

        img_dst = os.path.join(target_dir, "images", subset, file)
        lbl_dst = os.path.join(target_dir, "labels", subset, base + ".txt")

        shutil.copy2(img_src, img_dst)
        shutil.copy2(lbl_src, lbl_dst)

# Move them
move_files(train_files, "train")
move_files(val_files, "val")


In [ ]:
import albumentations as A
import cv2
import os
import shutil

augmentor = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=10, p=0.4),
    A.Blur(blur_limit=3, p=0.1),
    A.RandomScale(scale_limit=0.1, p=0.4)
])

knife_img_dir = '/content/weapons-dataset/images/train'
knife_lbl_dir = '/content/weapons-dataset/labels/train'

knife_files = [f for f in os.listdir(knife_lbl_dir) if open(os.path.join(knife_lbl_dir, f)).read().startswith("0")]

for f in knife_files:
    base = f.replace('.txt', '')
    img_path = os.path.join(knife_img_dir, base + '.jpg')  # or .png
    label_path = os.path.join(knife_lbl_dir, f)

    if not os.path.exists(img_path):
        continue  # skip missing images

    img = cv2.imread(img_path)
    for i in range(6):  # make 6 copies
        aug = augmentor(image=img)
        aug_img = aug['image']

        new_name = f"{base}_aug{i}"
        cv2.imwrite(os.path.join(knife_img_dir, new_name + '.jpg'), aug_img)
        shutil.copy(label_path, os.path.join(knife_lbl_dir, new_name + '.txt'))


In [ ]:
# !python train.py --img 640 --batch 16 --epochs 50 \
# --data /content/yolov5/weapons-dataset/dataset.yaml \
# --weights yolov5s.pt --cache
# !python train.py --img 416 --batch 8 --epochs 10 \
# --data /content/yolov5/weapons-dataset/dataset.yaml \
# --weights yolov5s.pt
!python train.py --img 640 --batch 8 --epochs 20 \
--data /content/yolov5/weapons-dataset/dataset.yaml \
--weights yolov5m.pt --patience 5


Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
      18/19      4.65G     0.0366    0.01547  0.0005718         23        640:  45% 721/1605 [03:13<03:56,  3.74it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      18/19      4.65G    0.03659    0.01546  0.0005712         15        640:  45% 722/1605 [03:13<03:52,  3.79it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      18/19      4.65G    0.03659    0.01546  0.0005705         16        640:  45% 723/1605 [03:13<03:40,  4.00it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autoca

In [ ]:
import os

label_dir = "/content/weapons-dataset/labels/train"
class_ids = set()

for file in os.listdir(label_dir):
    if file.endswith(".txt"):
        with open(os.path.join(label_dir, file), "r") as f:
            for line in f:
                class_id = int(line.strip().split()[0])
                class_ids.add(class_id)

class_ids = sorted(class_ids)
print("Class IDs found:", class_ids)
print("Total classes:", len(class_ids))


Class IDs found: [0, 1]
Total classes: 2


In [ ]:
!python val.py --weights runs/train/exp/weights/best.pt --data /content/yolov5/weapons-dataset/dataset.yaml --task val


val: data=/content/yolov5/weapons-dataset/dataset.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 212 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
val: Scanning /content/weapons-dataset/labels/val.cache... 800 images, 0 backgrounds, 0 corrupt: 100% 800/800 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 25/25 [00:16<00:00,  1.55it/s]
                   all        800        889      0.749      0.729      0.726      0.441
                 Knife        800        423      0.668      0.619      0.581      0.234
   

In [ ]:
from google.colab import files
files.download('/content/yolov5/runs/train/exp/weights/best.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cat /content/yolov5/weapons-dataset/dataset/dataset/labels/train/*.txt | cut -d' ' -f1 | sort | uniq -c


    381 0
   2081 1


In [ ]:
!cat /content/weapons-dataset/labels/train/*.txt | cut -d' ' -f1 | sort | uniq -c

   1206 0
   1654 1
